In [2]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

In [3]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 15  # Any arbitrary number
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape

# Make a lstm cell with hidden_size (each unit output vector size)

cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)

multi_cells = rnn.MultiRNNCell([cell] * 6, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

#softmax 
X_for_softmax = tf.reshape(outputs,[-1, hidden_size])
softmax_w = tf.get_variable("softmax_w", [hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b 

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])


# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

W0926 00:03:03.874421   848 deprecation.py:323] From <ipython-input-3-5a2057e0c29e>:38: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0926 00:03:03.875422   848 deprecation.py:323] From <ipython-input-3-5a2057e0c29e>:40: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0926 00:03:03.876419   848 rnn_cell_impl.py:1642] At least two cells provided to MultiRNNCell are the same object and will share weights.
W0926 00:03:03.877419   848 deprecation.py:323] From <ipython-input-3-5a2057e0c29e>:43: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future 

0 if you want to  -> f you want to b
1 f you want to b ->  you want to bu
2  you want to bu -> you want to bui
3 you want to bui -> ou want to buil
4 ou want to buil -> u want to build
5 u want to build ->  want to build 
6  want to build  -> want to build a
7 want to build a -> ant to build a 
8 ant to build a  -> nt to build a s
9 nt to build a s -> t to build a sh
10 t to build a sh ->  to build a shi
11  to build a shi -> to build a ship
12 to build a ship -> o build a ship,
13 o build a ship, ->  build a ship, 
14  build a ship,  -> build a ship, d
15 build a ship, d -> uild a ship, do
16 uild a ship, do -> ild a ship, don
17 ild a ship, don -> ld a ship, don'
18 ld a ship, don' -> d a ship, don't
19 d a ship, don't ->  a ship, don't 
20  a ship, don't  -> a ship, don't d
21 a ship, don't d ->  ship, don't dr
22  ship, don't dr -> ship, don't dru
23 ship, don't dru -> hip, don't drum
24 hip, don't drum -> ip, don't drum 
25 ip, don't drum  -> p, don't drum u
26 p, don't drum u -> 

W0926 00:03:04.470150   848 deprecation.py:506] From W:\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0926 00:03:04.490138   848 deprecation.py:506] From W:\Anaconda3\lib\site-packages\tensorflow\python\ops\rnn_cell_impl.py:738: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


0 0 fffffffffffffff 3.2923234
0 1 fffffffffffffff 3.2923234
0 2 fffffffffffffff 3.2923234
0 3 fffffffffffffff 3.2923234
0 4 fffffffffffffff 3.2923234
0 5 fffffffffffffff 3.2923234
0 6 fffffffffffffff 3.2923234
0 7 fffffffffffffff 3.2923234
0 8 fffffffffffffff 3.2923234
0 9 fffffffffffffff 3.2923234
0 10 fffffffffffffff 3.2923234
0 11 fffffffffffffff 3.2923234
0 12 fffffffffffffff 3.2923234
0 13 fffffffffffffff 3.2923234
0 14 fffffffffffffff 3.2923234
0 15 fffffffffffffff 3.2923234
0 16 fffffffffffffff 3.2923234
0 17 fffffffffffffff 3.2923234
0 18 fffffffffffffff 3.2923234
0 19 fffffffffffffff 3.2923234
0 20 fffffffffffffff 3.2923234
0 21 fffffffffffffff 3.2923234
0 22 fffffffffffffff 3.2923234
0 23 fffffffffffffff 3.2923234
0 24 fffffffffffffff 3.2923234
0 25 fffffffffffffff 3.2923234
0 26 fffffffffffffff 3.2923234
0 27 fffffffffffffff 3.2923234
0 28 fffffffffffffff 3.2923234
0 29 fffffffffffffff 3.2923234
0 30 fffffffffffffff 3.2923234
0 31 fffffffffffffff 3.2923234
0 32 fffffffffffff

2 0 ooooooooooooooo 3.111427
2 1 ooooooooooooooo 3.111427
2 2 ooooooooooooooo 3.111427
2 3 ooooooooooooooo 3.111427
2 4 ooooooooooooooo 3.111427
2 5 ooooooooooooooo 3.111427
2 6 ooooooooooooooo 3.111427
2 7 ooooooooooooooo 3.111427
2 8 ooooooooooooooo 3.111427
2 9 ooooooooooooooo 3.111427
2 10 ooooooooooooooo 3.111427
2 11 ooooooooooooooo 3.111427
2 12 ooooooooooooooo 3.111427
2 13 ooooooooooooooo 3.111427
2 14 ooooooooooooooo 3.111427
2 15 ooooooooooooooo 3.111427
2 16 ooooooooooooooo 3.111427
2 17 ooooooooooooooo 3.111427
2 18 ooooooooooooooo 3.111427
2 19 ooooooooooooooo 3.111427
2 20 ooooooooooooooo 3.111427
2 21 ooooooooooooooo 3.111427
2 22 ooooooooooooooo 3.111427
2 23 ooooooooooooooo 3.111427
2 24 ooooooooooooooo 3.111427
2 25 ooooooooooooooo 3.111427
2 26 ooooooooooooooo 3.111427
2 27 ooooooooooooooo 3.111427
2 28 ooooooooooooooo 3.111427
2 29 ooooooooooooooo 3.111427
2 30 ooooooooooooooo 3.111427
2 31 ooooooooooooooo 3.111427
2 32 ooooooooooooooo 3.111427
2 33 ooooooooooooooo

...

491 31 dt arum up peop 0.17175274
491 32 t arum up peopl 0.17175274
491 33  drum up people 0.17175274
491 34 toum up people  0.17175274
491 35  um up people t 0.17175274
491 36  m up people to 0.17175274
491 37 t up people tog 0.17175274
491 38  up people toge 0.17175274
491 39 tp people toget 0.17175274
491 40 t people togeth 0.17175274
491 41  people togethe 0.17175274
491 42 teople together 0.17175274
491 43  ople together  0.17175274
491 44 rple together t 0.17175274
491 45 nle together to 0.17175274
491 46  e together to  0.17175274
491 47 e together to c 0.17175274
491 48 rtogether to co 0.17175274
491 49 to ether to col 0.17175274
491 50   ether to coll 0.17175274
491 51 nether to colle 0.17175274
491 52  ther to collec 0.17175274
491 53 rher to collect 0.17175274
491 54  em te collect  0.17175274
491 55 em te collect w 0.17175274
491 56 r to collect wo 0.17175274
491 57  to collect woo 0.17175274
491 58 to bollect wood 0.17175274
491 59   collect wood  0.17175274
491 60 ncollec

493 41  people togethe 0.16928382
493 42 teople together 0.16928382
493 43  ople together  0.16928382
493 44  ple together t 0.16928382
493 45  le together to 0.16928382
493 46  e together to  0.16928382
493 47 estogether to c 0.16928382
493 48  together to co 0.16928382
493 49 to ether to col 0.16928382
493 50   ether to coll 0.16928382
493 51  ether to colle 0.16928382
493 52  ther to collec 0.16928382
493 53  her to collect 0.16928382
493 54  e  to collect  0.16928382
493 55 e  to collect w 0.16928382
493 56   te collect wo 0.16928382
493 57  to collect woo 0.16928382
493 58 to lollect wood 0.16928382
493 59   lollect wood  0.16928382
493 60  lollect wood a 0.16928382
493 61 tollect wood an 0.16928382
493 62 ellect wood and 0.16928382
493 63  lect wood and  0.16928382
493 64 eect wood and d 0.16928382
493 65 est wood and do 0.16928382
493 66  t wood and don 0.16928382
493 67 e wood and don' 0.16928382
493 68  dood and don't 0.16928382
493 69 tood and don't  0.16928382
493 70 ood and

495 0 p you want to b 0.17574605
495 1  you want to bu 0.17574605
495 2 tou want to bui 0.17574605
495 3 ou want to buil 0.17574605
495 4   want to build 0.17574605
495 5 twant to build  0.17574605
495 6 tant to build a 0.17574605
495 7 ont to build a  0.17574605
495 8 nd to build a s 0.17574605
495 9 d to build a sh 0.17574605
495 10 hdo build a shi 0.17574605
495 11 to luild a ship 0.17574605
495 12 h luild a ship, 0.17574605
495 13  cuild a ship,  0.17574605
495 14 tutld a ship, d 0.17574605
495 15 utld a ship, do 0.17574605
495 16 tld a ship, don 0.17574605
495 17 pd a ship, don' 0.17574605
495 18 e a ship, don't 0.17574605
495 19  d ship, don't  0.17574605
495 20 t ship, don't d 0.17574605
495 21 nship, don't dr 0.17574605
495 22 thip, don't dru 0.17574605
495 23  ip, don't drum 0.17574605
495 24 ep, don't drum  0.17574605
495 25 p, don't drum u 0.17574605
495 26 t don't drum up 0.17574605
495 27  don't drum up  0.17574605
495 28 ton't drum up p 0.17574605
495 29  n't drum up pe 0

497 0 p you want to b 0.16979986
497 1  you want to bu 0.16979986
497 2 tou want to bui 0.16979986
497 3 ou want to buil 0.16979986
497 4   want to build 0.16979986
497 5 twant to build  0.16979986
497 6 tont to build a 0.16979986
497 7 ont to build a  0.16979986
497 8 nd to build a s 0.16979986
497 9 d to build a sh 0.16979986
497 10  do build a shi 0.16979986
497 11 to build a ship 0.16979986
497 12   build a ship, 0.16979986
497 13  build a ship,  0.16979986
497 14 tutld a ship, d 0.16979986
497 15 utld a ship, do 0.16979986
497 16 tld a ship, don 0.16979986
497 17 pd a ship, don' 0.16979986
497 18 e a ship, don't 0.16979986
497 19  a ship, don't  0.16979986
497 20 t ship, don't d 0.16979986
497 21 nship, don't dr 0.16979986
497 22 thip, don't dru 0.16979986
497 23  ip, don't drum 0.16979986
497 24 ep, don't drum  0.16979986
497 25 p, don't drum u 0.16979986
497 26   don't drum up 0.16979986
497 27  don't drum up  0.16979986
497 28 ton't drum up p 0.16979986
497 29  n't drum up pe 0

499 47 e together to c 0.17071995
499 48  together to co 0.17071995
499 49 th ether to col 0.17071995
499 50   ether to coll 0.17071995
499 51  ether to colle 0.17071995
499 52  ther to collec 0.17071995
499 53  her to collect 0.17071995
499 54  em to collect  0.17071995
499 55 em to collect w 0.17071995
499 56   to collect wo 0.17071995
499 57  to collect woo 0.17071995
499 58 th bollect wood 0.17071995
499 59   bollect wood  0.17071995
499 60  bollect wood a 0.17071995
499 61 tollect wood an 0.17071995
499 62 ellect wood and 0.17071995
499 63  lect wood and  0.17071995
499 64 eect wood and d 0.17071995
499 65 e t wood and do 0.17071995
499 66  t wood and don 0.17071995
499 67 e wood and don' 0.17071995
499 68  dood and don't 0.17071995
499 69 tood and don't  0.17071995
499 70 ood and don't a 0.17071995
499 71  d and don't as 0.17071995
499 72   and don't ass 0.17071995
499 73  a d don't assi 0.17071995
499 74 tnd don't assig 0.17071995
499 75 nd don't assign 0.17071995
499 76 d don't

In [ ]:
"if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")